Capstone-Project

FindDefault (Prediction of Credit Card fraud)

Problem Statement:

A credit card is one of the most used financial products to make online purchases and payments. Though the Credit cards can be a convenient way to manage your finances, they can also be risky. Credit card fraud is the unauthorized use of someone else's credit card or credit card information to make purchases or withdraw cash.
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase. 
The dataset contains transactions made by credit cards in September 2013 by European cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
We have to build a classification model to predict whether a transaction is fraudulent or not.


Tasks/Activities List 

	Collect the time series data from the CSV file linked here

	Exploratory Data Analysis (EDA) 

	Get the correct datatype for date.

	Balancing the data.

	Feature Engineering and feature selection.

	Train/Test Split - Apply a sampling distribution to find the best split

	Choose the metrics for the model evaluation

	Model Selection, Training, Predicting and Assessment 

	Hyperparameter Tuning/Model Improvement 

	Model deployment plan. 


In [98]:
# Load Required Libraries

import pandas as pd
import numpy as np

# EDA analysis 

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import show
from matplotlib.colors import ListedColormap
#from plotnine import *
import matplotlib.pyplot as Pyplot

from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats import shapiro
import sklearn.metrics as metrics
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,validation_curve, StratifiedShuffleSplit,StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.datasets import make_classification


from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn import over_sampling

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
import warnings
warnings.warn('foo', DeprecationWarning)
import warnings, sklearn.utils
warnings.warn('bar', DeprecationWarning)

In [ ]:
# Will create the seperator for better data visualization among each variables
def Line_Separator():
    print('*'*50, '\n')
    
def Line_Separator1():
    print('*'*100, '\n')

Read The Dataset

In [ ]:
creditcard = pd.read_csv('creditcard csv')

In [ ]:
creditcard.head()

In [ ]:
# Evaluate number of columns and rows in given dataset

Number_of_row = creditcard.shape[0]
Number_of_column = creditcard.shape[1]

print('Number of rows in creditcard file     :', Number_of_row)
print('Number of columns in creditcard file  :', Number_of_column); Line_Separator()

In [ ]:
# Review columns title in given dataset

print("Columns name in  creditcard file :\n",creditcard.columns.values);Line_Separator1()

In [ ]:
# Evaluate number of categorical and numerical features

def data_features (data):
    categorical_features = creditcard.select_dtypes(exclude = [np.number]).columns
    numerical_features = creditcard.select_dtypes(include = [np.number]).columns
    print("Categorical features in  creditcard file :\n",categorical_features);Line_Separator1()
    print("Numerical features in  creditcard file   :\n",numerical_features);Line_Separator1()
    
print(data_features(creditcard))

In [ ]:
# Review the datatypes
print("Review the Data Format in  creditcard file :");Line_Separator()
print(creditcard.dtypes);Line_Separator()

In [ ]:
# Identify missing value if any

print("Check if there is any missing value in in  creditcard file :");Line_Separator()
print(round(100*(creditcard.isnull()).sum()/len(creditcard),2).sort_values(ascending=False));Line_Separator()

We are check Missing values, but i have not find any missing values

check summary statistics credit card time column

In [ ]:
print("Time variable statistics");Line_Separator()
creditcard['Time'].describe()

In [ ]:
# Data is 48hours, and the values seems to represent the second will convert it to hours - 1hour =3600seconds 

creditcard['Time'] =creditcard['Time']/3600

In [ ]:
# Assignment of classes one and zero for visualization 

def replace_data_to_binary(x,y):
    creditcard.Class.replace(x,y, inplace=True)
  
    
replace_data_to_binary(0, 'Non Fraudulent')
replace_data_to_binary(1, 'Fraudulent')

In [ ]:
# Readmitted distribution

fig = plt.figure(figsize=(9,5))
fig.set_facecolor("#F3F3F3")
total = float(len(creditcard)) 
ax = sns.countplot(x="Class",  data=creditcard)
plt.xlabel('Class', fontsize=14)
plt.ylabel('Number of observations', fontsize=14)
plt.title('Distrinution of Class', fontsize =14)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2., height + 3, '{:1.2f}'.format(100*height/total), 
            ha="center",va='bottom')
ax.grid(False)
plt.tick_params(labelsize=12)
plt.show();Line_Separator1()

In [ ]:
# Evaluate whether data is balanced or not 

total_count_combined_calss = creditcard['Class'].value_counts()
imbalance= (total_count_combined_calss['Fraudulent']/creditcard['Class'].count()*100)/(total_count_combined_calss['Non Fraudulent']/creditcard['Class'].count()*100)*100
print('Imbalance Percentage : ' + str(imbalance));Line_Separator()

In [ ]:
# 1. Time vs Amount scatter plot
plt.figure(figsize=(12, 6))
sns.scatterplot(data=creditcard, x='Time', y='Amount', hue='Class', palette='viridis', alpha=0.6)
plt.title('Time vs Amount Scatter Plot')
plt.xlabel('Time')
plt.ylabel('Amount')
plt.legend(title='Class')
plt.show()

In [ ]:
print("Amount details of fraudulent transactions:"); Line_Separator()
print(creditcard[creditcard["Class"] == "Fraudulent"].Amount.describe()); Line_Separator1()

print('\n')

print("Amount details of non-fraudulent transactions:"); Line_Separator()
print(creditcard[creditcard["Class"] == "Non Fraudulent"].Amount.describe()); Line_Separator1()


In [ ]:
# Evaluate the time vs. amount transaction between fraudulent and non-fraudulent

print("Evaluate the time vs. amount transaction between fraudulent and non-fraudulent");Line_Separator1()
ax=sns.relplot(x="Time", y="Amount",
                 col="Class", hue="Class",
                 kind="scatter", data=creditcard)
plt.tick_params(labelsize=12)
plt.show(); Line_Separator1()

Few insights on the visualization above reveal the following:

The plot indicates that the fraud amounts were less than approx 2.2k.
Fraud pattern indicates that the number of data points is observed between 14 to 20 hours on both days.
We can see a two-picks pattern in time due tonight.

In [ ]:
#To assign back the values 'Non Fraudulent' and 'Fraudulent' back to 0 and 1 in the Class
def replace_data_to_binary(x, y):
    creditcard['Class'].replace(x, y, inplace=True)

# Replace 'Non Fraudulent' with 0 and 'Fraudulent' with 1
replace_data_to_binary('Non Fraudulent', 0)
replace_data_to_binary('Fraudulent', 1)

In [ ]:
print("After reassigning class to zero and one, we will evaluate the data type ");Line_Separator1()
print(creditcard.dtypes)


In [ ]:
# 3. Correlation heatmap
plt.figure(figsize=(16, 12))
correlation_matrix = creditcard.corr()
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Evaluate the data distributions of each variable

print(); Line_Separator1()
print("Plotting the Shape of a Distribution of each Variable in 'Non-Fraudulent' & 'Fraudulent': Variables V1 to V12"); Line_Separator1()
fig = plt.figure(figsize=(20,15))
fig.set_facecolor("#F3F3F3")

plt.subplot(431)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V1'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V1'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V1", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(432)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V2'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V2'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V2", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(433)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V3'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V3'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V3", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(434)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V4'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V4'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V4", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(435)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V5'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V5'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V5", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(436)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V6'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V6'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V6", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(437)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V7'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V7'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V7", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(438)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V8'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V8'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V8", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(439)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V9'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V9'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V9", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(4,3,10)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V10'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V10'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V10", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(4,3,11)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V11'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V11'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V11", fontsize=12)
plt.tick_params(labelsize=12)

plt.subplot(4,3,12)
g=sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0),'V12'] , color='b',shade=True,label='Non Fraudulent')
g=sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1),'V12'] , color='orange',shade=True, label='Fraudulent')
g.grid(False)
plt.ylabel("Frequency Density", fontsize=12)
plt.xlabel("V12", fontsize=12)
plt.tick_params(labelsize=12)

plt.show(); Line_Separator1()

In [ ]:
print(); Line_Separator1()
print("Plotting the Shape of a Distribution of each Variable in 'Non-Fraudulent' & 'Fraudulent': Variables V13 to V24"); Line_Separator1()

fig = plt.figure(figsize=(20,15))
fig.set_facecolor("#F3F3F3")

variables = ['V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']
colors = ['b', 'orange']

for i, var in enumerate(variables):
    plt.subplot(4, 3, i+1)
    sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0), var], color=colors[0], shade=True, label='Non Fraudulent')
    sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1), var], color=colors[1], shade=True, label='Fraudulent')
    plt.grid(False)
    plt.ylabel("Frequency Density", fontsize=12)
    plt.xlabel(var, fontsize=12)
    plt.tick_params(labelsize=12)

plt.show(); Line_Separator1()


In [ ]:
print(); Line_Separator1()
print("Plotting the Shape of a Distribution of each Variable in 'Non-Fraudulent' & 'Fraudulent': Variables V25 to V28, Time & Amount"); Line_Separator1()

fig = plt.figure(figsize=(20,15))
fig.set_facecolor("#F3F3F3")

# List of variables to plot
variables = ['V25', 'V26', 'V27', 'V28', 'Time', 'Amount']
colors = ['b', 'orange']

# Plot each variable in a subplot
for i, var in enumerate(variables):
    plt.subplot(4, 3, i + 1)
    sns.kdeplot(creditcard.loc[(creditcard["Class"] == 0), var], color=colors[0], shade=True, label='Non Fraudulent')
    sns.kdeplot(creditcard.loc[(creditcard['Class'] == 1), var], color=colors[1], shade=True, label='Fraudulent')
    plt.grid(False)
    plt.ylabel("Frequency Density", fontsize=12)
    plt.xlabel(var, fontsize=12)
    plt.tick_params(labelsize=12)

plt.tight_layout()
plt.show(); Line_Separator1()


In [ ]:
#  Histograms of the features
creditcard.hist(figsize=(20, 20), bins=50)
plt.suptitle('Histograms of All Features')
plt.show()

In [ ]:
# Separate data of Non-Fraudulent and Fraudulent to check the skewness and kurtosis

Non_Fraudulent= creditcard[creditcard["Class"] == 0]
print ("Non_Fraudulent:", Non_Fraudulent.shape); Line_Separator()
Fraudulent= creditcard[creditcard["Class"] == 1]
print ("Fraudulent:", Fraudulent.shape); Line_Separator()

In [ ]:
# Non_Fraudulent = to check the skewness and kurtosis.

print ("Non_Fraudulent = To evaluate Mean, Variance, skewness, and kurtosis:")
print("")
a = Non_Fraudulent.mean(axis = 0, skipna = True)
b = Non_Fraudulent.var(axis = 0, skipna = True)
c = Non_Fraudulent.skew(axis = 0, skipna = True)
d = Non_Fraudulent.kurtosis(axis = 0, skipna = True)

a.index = b.index
a.index = c.index
a.index = d.index

data_Non_Fraudulent = pd.concat([a, b, c, d] ,axis = 1)
data_Non_Fraudulent.columns = ["Mean", "Var", "Skewness", "kurtosis"]
data_Non_Fraudulent=data_Non_Fraudulent.reset_index().rename(index=str, columns={"index": "Variables"})
print(data_Non_Fraudulent); Line_Separator1()

# -------------------------------------------------------------------------------------------------------------

# Fraudulent = to check the skewness and kurtosis

print ("Fraudulent = To evaluate Mean, Variance, skewness, and kurtosis:")
print("")
e = Fraudulent.mean(axis = 0, skipna = True)
f = Fraudulent.var(axis = 0, skipna = True)
g = Fraudulent.skew(axis = 0, skipna = True)
h = Fraudulent.kurtosis(axis = 0, skipna = True)

e.index = f.index
e.index = g.index
e.index = h.index

data_Fraudulent = pd.concat([a, b, c, d] ,axis = 1)
data_Fraudulent.columns = ["Mean", "Var", "Skewness", "kurtosis"]
data_Fraudulent=data_Fraudulent.reset_index().rename(index=str, columns={"index": "Variables"})
print(data_Fraudulent); Line_Separator1()

1.Skewness = 0 : normally distributed. ; a zero value means that the tails on both sides of the mean balance out overall,
2.Skewness > 0: more weight in the left tail of the distribution.
3.Skewness < 0: more weight in the right tail of the distribution.
For example, a zero value means that the tails on both sides of the mean balance out overall; this is the case for asymmetric distribution, but it can also be true for an asymmetric distribution where one tail is long and thin, and the other is short but fat.

Note: we will look into this if we require any power transformation to the data. However, they are PCA transformed (not an original one)

In [ ]:
# Non-Fraudulent: Evaluate the number of positive skewness variables 

print('Non Fraudulent - Positive skewness:')
left_skewness_Non_Fraudulent= data_Non_Fraudulent[data_Non_Fraudulent.Skewness >0]
print(left_skewness_Non_Fraudulent['Variables'].unique());Line_Separator()


In [ ]:
# Fraudulent : Evaluate the number of positive skewness variables

print('Fraudulent - Positive skewness:')
left_skewness_Fraudulent = data_Fraudulent[data_Fraudulent.Skewness >0]
print(left_skewness_Fraudulent['Variables'].unique());Line_Separator()

Positive Skew: Mean > median

In [ ]:
# Non-Fraudulent: Evaluate the number of negative skewness variables 
print('Non Fraudulent - Negative skewness:')
right_skewness_Non_Fraudulent= data_Non_Fraudulent[data_Non_Fraudulent.Skewness <0]
print(right_skewness_Non_Fraudulent['Variables'].unique());Line_Separator()

# ---------------------------------------------------------------------------------------

# Fraudulent: Evaluate the number of negative skewness variables 
print('Fraudulent - Negative skewness:')
right_skewness_Fraudulent = data_Fraudulent[data_Fraudulent.Skewness <0]
print(right_skewness_Fraudulent['Variables'].unique());Line_Separator()

Negative Skew: Median > mean
Zero Skew: Mean = median (normal distribution)

In [ ]:
# We will drop the time column

creditcard = creditcard.drop(['Time'],axis=1)
creditcard.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

creditcard['Amount'] = StandardScaler().fit_transform(creditcard['Amount'].values.reshape(-1,1)) 
#creditcard['Time'] = StandardScaler().fit_transform(creditcard['Time'].values.reshape(-1,1)) 

In [ ]:
# Create X and y

X = creditcard.drop('Class',axis=1)
y = creditcard['Class']

In [ ]:
# Evaluate the X dataset

X.head(5)

In [ ]:
# print the data shape

print(X.shape)
print(y.shape)

Splitting the dataset to Train and Test¶

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3,train_size=0.7,random_state=0)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Apply power transformation using 'yeo-johnson' method
creditcard_pt = PowerTransformer(method='yeo-johnson', copy=True)
creditcard_pt.fit(X_train)

# Transform the training and testing data
X_train_pt = creditcard_pt.transform(X_train)
X_test_pt = creditcard_pt.transform(X_test)

# Assign y_train to y_train_pt (No transformation needed for target variable)
y_train_pt = y_train

# Check the transformed data
print("Transformed X_train:")
print(X_train_pt)
print("\nTransformed X_test:")
print(X_test_pt)


In [ ]:
# Rename the x_train and x_test

X_train = X_train_pt
X_test = X_test_pt


In [ ]:
from imblearn.over_sampling import ADASYN
from collections import Counter

# Apply ADASYN oversampling
adasyn = ADASYN(sampling_strategy='minority', random_state=42)
X_ada, y_ada = adasyn.fit_resample(X_train, y_train)

# Print the resampled dataset shape
print('Resampled dataset shape %s' % Counter(y_ada))


In [ ]:
# Shape of the dataset after balancing it
print(X_ada.shape)
print(y_ada.shape)

In [ ]:
# Evalute unique data in y_ada

y_ada.unique()

In [ ]:
from imblearn.over_sampling import ADASYN
from collections import Counter

# Apply ADASYN oversampling
adasyn = ADASYN(sampling_strategy='minority', random_state=42)
X_ada, y_ada = adasyn.fit_resample(X_train, y_train)

# Print the resampled dataset shape
print('Resampled dataset shape %s' % Counter(y_ada))

# Evaluate whether the dataset is balanced
total_count_combined_class = y_ada.value_counts()
imbalance = (total_count_combined_class[1] / y_ada.count() * 100) / (total_count_combined_class[0] / y_ada.count() * 100) * 100

print('Balance Percentage after ADASYN: ' + str(imbalance) + '%')
Line_Separator1()


In [ ]:
# Evaluate the sum of y_ada, y_train and y_test

print(np.sum(y_ada))
print(np.sum(y_train))
print(np.sum(y_test))

In [ ]:
# Rename X_ada and y_ada

X_train = X_ada
y_train = y_ada

In [ ]:
# Evaluate the shape of the training and test dataset after balancing it

print(X_train.shape)
print(y_train.shape)

Base Model

In [ ]:
# Base Model for evaluation

b_m=[]

for i in range (y_test.shape[0]):
    b_m.append(y_test.mode()[0])

len(b_m)

In [ ]:
y_pred=pd.Series(b_m)

In [ ]:
print('Accuracy (Base Model):', accuracy_score(y_test, y_pred)*100);Line_Separator()

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("ROC-AUC Score:", roc_auc_score(y_test, y_proba))

    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label="ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="best")
    plt.show()


Model Selection and Model Training

We will use Logistic Regression, K-Nearest Neighbors, Decision Tree, and XGBoost as our models.

1. Let's Create Logistic Model

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train, y_train)


In [ ]:
#Model Validation
print("Logistic Regression Evaluation:")
evaluate_model(lr, X_test_pt, y_test)

2.Let's Create KNN Model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [ ]:
#Model Validation
print("K-Nearest Neighbors Evaluation:")
evaluate_model(knn, X_test_pt, y_test)

3.Lets Create DecisionTreeClassifier Model

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)


In [ ]:
#Model Validation
print("Decision Tree Evaluation:")
evaluate_model(dt, X_test_pt, y_test)

4.Let's Create XGBClassifier Model

In [ ]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)

In [ ]:
#Model Validation
print("XGBoost Evaluation:")
evaluate_model(xgb, X_test_pt, y_test)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, classification_report, confusion_matrix

def get_model_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_proba)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    return f1, accuracy, auc_roc, precision, recall


In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Train and evaluate each model, storing the results
model_summary = []

for name, model in models.items():
    model.fit(X_train, y_train)
    f1, accuracy, auc_roc, precision, recall = get_model_metrics(model, X_test_pt, y_test)
    model_summary.append([name, f1, accuracy, auc_roc, precision, recall])

# Create a DataFrame for model summary
model_summary_df = pd.DataFrame(model_summary, columns=['Name', 'F1_score', 'Accuracy', 'AUC_ROC', 'Precision', 'Recall'])


In [ ]:
model_summary_df = model_summary_df.reset_index(drop=True)
display(model_summary_df)

All Model Classification Report

In [ ]:
# Evaluation and Classification Reports
# Logistic Regression
print("Logistic Regression - Classification Report")
print(classification_report(y_test, lr.predict(X_test_pt)))

# K-Nearest Neighbors
print("K-Nearest Neighbors - Classification Report")
print(classification_report(y_test, knn.predict(X_test_pt)))

# Decision Tree
print("Decision Tree Evaluation - Classification Report")
print(classification_report(y_test, dt.predict(X_test_pt)))

# XGBoost
print("XGBoost - Classification Report")
print(classification_report(y_test, xgb.predict(X_test_pt)))


In [ ]:
# Confusion matrices
# Logistic Regression
lr_pred = lr.predict(X_test_pt)
lr_cm = confusion_matrix(y_test, lr_pred)

# K-Nearest Neighbors
knn_pred = knn.predict(X_test_pt)
knn_cm = confusion_matrix(y_test, knn_pred)

# DT
dt_pred = dt.predict(X_test_pt)
dt_cm = confusion_matrix(y_test, dt_pred)

# XGBoost
xgb_pred = xgb.predict(X_test_pt)
xgb_cm = confusion_matrix(y_test, xgb_pred)

# Plotting confusion matrices
fig, axes = plt.subplots(1, 4, figsize=(24, 5))
fig.suptitle('Confusion Matrices', fontsize=16)

# Logistic Regression
sns.heatmap(lr_cm, annot=True, cmap='Blues', fmt='d', cbar=False, ax=axes[0])
axes[0].set_title('Logistic Regression')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')

# K-Nearest Neighbors
sns.heatmap(knn_cm, annot=True, cmap='Blues', fmt='d', cbar=False, ax=axes[1])
axes[1].set_title('K-Nearest Neighbors')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('True')

#dt
sns.heatmap(dt_cm, annot=True, cmap='Blues', fmt='d', cbar=False, ax=axes[2])
axes[2].set_title('Decision Tree')
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('True')

# XGBoost
sns.heatmap(xgb_cm, annot=True, cmap='Blues', fmt='d', cbar=False, ax=axes[3])
axes[3].set_title('XGBoost')
axes[3].set_xlabel('Predicted')
axes[3].set_ylabel('True')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming you have computed confusion matrices: lr_cm, knn_cm, xgb_cm

# Function to calculate metrics
def calculate_metrics(cm):
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]
    
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    FPR = FP / (FP + TN)
    PPV = TP / (TP + FP)
    NPV = TN / (TN + FN)
    
    return sensitivity, specificity, FPR, PPV, NPV

# Calculate metrics for Logistic Regression
lr_sensitivity, lr_specificity, lr_fpr, lr_ppv, lr_npv = calculate_metrics(lr_cm)

# Calculate metrics for K-Nearest Neighbors
knn_sensitivity, knn_specificity, knn_fpr, knn_ppv, knn_npv = calculate_metrics(knn_cm)

# Calculate metrics for Decision Tree
dt_sensitivity, dt_specificity, dt_fpr, dt_ppv, dt_npv = calculate_metrics(dt_cm)

# Calculate metrics for XGBoost
xgb_sensitivity, xgb_specificity, xgb_fpr, xgb_ppv, xgb_npv = calculate_metrics(xgb_cm)

# Print metrics for each model
print("Logistic Regression Metrics:")
print(f"Sensitivity: {lr_sensitivity:.4f}, Specificity: {lr_specificity:.4f}, FPR: {lr_fpr:.4f}, PPV: {lr_ppv:.4f}, NPV: {lr_npv:.4f}")

print("\nK-Nearest Neighbors Metrics:")
print(f"Sensitivity: {knn_sensitivity:.4f}, Specificity: {knn_specificity:.4f}, FPR: {knn_fpr:.4f}, PPV: {knn_ppv:.4f}, NPV: {knn_npv:.4f}")

print("\nDecision Tree Metrics:")
print(f"Sensitivity: {dt_sensitivity:.4f}, Specificity: {dt_specificity:.4f}, FPR: {dt_fpr:.4f}, PPV: {dt_ppv:.4f}, NPV: {dt_npv:.4f}")

print("\nXGBoost Metrics:")
print(f"Sensitivity: {xgb_sensitivity:.4f}, Specificity: {xgb_specificity:.4f}, FPR: {xgb_fpr:.4f}, PPV: {xgb_ppv:.4f}, NPV: {xgb_npv:.4f}")


# ROC - Curves for all the models

In [ ]:
# Initialize models
lr = LogisticRegression(max_iter=1000, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
xgb = XGBClassifier(random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# Fit models
lr.fit(X_train, y_train)
knn.fit(X_train, y_train)
xgb.fit(X_train, y_train)
dt.fit(X_train, y_train)

# Predict probabilities for ROC curve
lr_probs = lr.predict_proba(X_test)[:, 1]
knn_probs = knn.predict_proba(X_test)[:, 1]
xgb_probs = xgb.predict_proba(X_test)[:, 1]
dt_probs = dt.predict_proba(X_test)[:, 1]

# Calculate ROC curve and AUC
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
lr_roc_auc = auc(lr_fpr, lr_tpr)

knn_fpr, knn_tpr, _ = roc_curve(y_test, knn_probs)
knn_roc_auc = auc(knn_fpr, knn_tpr)

xgb_fpr, xgb_tpr, _ = roc_curve(y_test, xgb_probs)
xgb_roc_auc = auc(xgb_fpr, xgb_tpr)

dt_fpr, dt_tpr, _ = roc_curve(y_test, dt_probs)
dt_roc_auc = auc(dt_fpr, dt_tpr)

# Plot ROC curves
plt.figure(figsize=(10, 8))
plt.plot(lr_fpr, lr_tpr, color='blue', lw=2, label=f'LR ROC curve (area = {lr_roc_auc:.2f})')
plt.plot(knn_fpr, knn_tpr, color='green', lw=2, label=f'KNN ROC curve (area = {knn_roc_auc:.2f})')
plt.plot(xgb_fpr, xgb_tpr, color='red', lw=2, label=f'XGB ROC curve (area = {xgb_roc_auc:.2f})')
plt.plot(dt_fpr, dt_tpr, color='purple', lw=2, label=f'DT ROC curve (area = {dt_roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

Hyperparameter Tuning for Logistic Model

In [ ]:
from sklearn.model_seldection import GridSearchCV

# Hyperparameter tuning for Logistic Regression
param_grid_lr = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs']
}

grid_lr = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid_lr, cv=5, scoring='roc_auc')
grid_lr.fit(X_train, y_train)
print("Best parameters for Logistic Regression:", grid_lr.best_params_)
best_lr = grid_lr.best_estimator_
print("Tuned Logistic Regression Evaluation:")
evaluate_model(best_lr, X_test_pt, y_test)

Hyperparameter Tuning for Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define the parameter grid for Decision Tree
dt_param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Initialize the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
dt_grid_search = GridSearchCV(estimator=dt, param_grid=dt_param_grid, 
                              cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the model
dt_grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_dt = dt_grid_search.best_estimator_
print(f"Best Parameters for Decision Tree: {dt_grid_search.best_params_}")
print(f"Best Score for Decision Tree: {dt_grid_search.best_score_}")

# Evaluate the best model on the test set
dt_best_pred = best_dt.predict(X_test_pt)
print(classification_report(y_test, dt_best_pred))


Hyperparameter Tuning for XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Define the parameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.3, 0.7, 1.0],
    'subsample': [0.7, 1.0]
}

# Initialize the XGBClassifier
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Initialize GridSearchCV
xgb_grid_search = GridSearchCV(estimator=xgb, param_grid=xgb_param_grid, 
                               cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the model
xgb_grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_xgb = xgb_grid_search.best_estimator_
print(f"Best Parameters for XGBoost: {xgb_grid_search.best_params_}")
print(f"Best Score for XGBoost: {xgb_grid_search.best_score_}")

# Evaluate the best model on the test set
xgb_best_pred = best_xgb.predict(X_test_pt)
print(classification_report(y_test, xgb_best_pred))


Model Deployment

In [ ]:
import joblib

# Save the model
joblib.dump(best_xgb, 'best_xgb_model.pkl')
joblib.dump(best_dt, 'best_dt_model.pkl')


In [ ]:
import joblib

# Save the model
joblib.dump(best_xgb, 'best_xgb_model.pkl')

In [ ]:
# Load the model
loaded_xgb = joblib.load('best_xgb_model.pkl')
loaded_dt = joblib.load('best_dt_model.pkl')

# Make predictions
new_predictions_xgb = loaded_xgb.predict(X_test)
new_predictions_dt = loaded_dt.predict(X_test)

# Evaluate loaded models
print("Loaded XGBoost Model Evaluation:")
evaluate_model(loaded_xgb, X_test, y_test)

print("\nLoaded Decision Tree Model Evaluation:")
evaluate_model(loaded_dt, X_test, y_test)


In [ ]:
import pickle
from xgboost import XGBClassifier
from sklearn.datasets import load_iris


In [ ]:
# Initialize and train the model
model = XGBClassifier()
model.fit(X_train, y_train)


In [ ]:
# Save the model to a file
with open('best_xgb_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# Later on, load the model from the file
with open('best_xgb_model.pkl', 'rb') as file:
    best_xgb = pickle.load(file)


In [ ]:
# Use the loaded model to make predictions
preds = best_xgb.predict(X_test)

In [ ]:
# Print the predictions
print(preds)